# Binary Knapsack Problem Experiments
In the binary knapsack problem you are given:
- a set of objects o1, o2, ..., ok
- their values v1, v2, ..., vk
- their weights w1, w2, ..., wk
The goal is to maximize the total value of the selected objects, subject to a weight constraint w

In [5]:
import numpy as np

## Representation and Toy Problem Generation

In [239]:
class KnapsackProblem:
    num_objects = None
    values = None
    weights = None
    capacity = None
    
    def __init__(self, num_objects: int) -> None:
        self.num_objects = num_objects
        self.values = 2 ** np.random.rand(num_objects)
        self.weights = 2 ** np.random.rand(num_objects)
        self.capacity = 0.25 * self.weights.sum()
            
    def __str__(self):
        return f'Knapsack Problem Object with N={self.num_objects}\n\n' \
            + f'Values: {self.values}\n\n' \
            + f'Weights: {self.weights}\n\n' \
            + f'Capacity: {self.capacity}\n'

In [240]:
kp = KnapsackProblem(10)
print(kp)

Knapsack Problem Object with N=10

Values: [1.02031414 1.17422714 1.07654056 1.0308779  1.32078789 1.10768761
 1.88507114 1.37503895 1.46176694 1.88989744]

Weights: [1.27063587 1.63976153 1.03778725 1.2285731  1.44393849 1.56722674
 1.29149573 1.40561332 1.14955549 1.59465366]

Capacity: 3.4073102945122757



## Solutions Representation
1) List of integers
2) A permutation of K numbers

In [287]:
class Individual:
    order = None
    alpha = None # probability of mutation
    def __init__(self, kp: KnapsackProblem, alpha: float=0.05) -> None:
        self.order = np.random.permutation(kp.num_objects)
        self.alpha = alpha
        
    def __str__(self) -> str:
        return f'{self.order}'

In [242]:
def fitness(kp: KnapsackProblem,
            individual: Individual) -> float:
    remaining_weight = kp.capacity
    total_value = 0

    for idx in individual.order:
        if kp.weights[idx] < remaining_weight:
            remaining_weight -= kp.weights[idx]
            total_value += kp.values[idx]

    return total_value

In [262]:
def in_knapsack(kp: KnapsackProblem,
               individual: Individual) -> float:
    remaining_weight = kp.capacity
    kpi = set()
    
    for idx in individual.order:
        if kp.weights[idx] < remaining_weight:
            remaining_weight -= kp.weights[idx]
            kpi.add(idx)

    return kpi

In [244]:
kp = KnapsackProblem(10)
ind = Individual(kp)
print(kp)
print(f'fitness of individual {ind}: {fitness(kp, ind):.4f}')

Knapsack Problem Object with N=10

Values: [1.79015174 1.43104726 1.57502839 1.18029802 1.14537119 1.03255504
 1.24111292 1.62622048 1.56721896 1.39146521]

Weights: [1.65762732 1.91110148 1.46807393 1.22444378 1.86036366 1.52195529
 1.68551903 1.02505224 1.16253941 1.48006226]

Capacity: 3.749184598059915

fitness of individual [4 3 0 8 1 9 5 2 6 7]: 2.3257


## Evolutionary Methods

In [457]:
def mutation(ind: Individual) -> Individual:
    if np.random.rand() > ind.alpha:
        i1 = np.random.randint(10)
        i2 = np.random.randint(10)
        ind.order[i1], ind.order[i2] = ind.order[i2], ind.order[i1]
        
    return ind


def recombination(kp: KnapsackProblem,
                  p1: Individual,
                  p2: Individual
                 ) -> Individual:
    
    set1 = in_knapsack(kp, p1)
    set2 = in_knapsack(kp, p2)
    
    # copy intersection to offspring
    offspring = set1.intersection(set2)
    
    # copy rest with probability of 50%
    for ind in set1.symmetric_difference(set2):
        if np.random.rand() <= 0.5:
            offspring.add(ind)
    
    order = list(range(kp.num_objects))
    i = 0
    for obj in offspring:
        order[i] = obj
        i += 1
    
    rem = set(range(kp.num_objects)).difference(offspring)
    for obj in rem:
        order[i] = obj
        i += 1
        
    # randomly permute the elements
    order[:len(offspring)] = [
        order[idx] for idx in np.random.permutation(
        range(len(offspring)))]
    
    order[len(offspring):] = [
        order[idx] for idx in np.random.permutation(
        range(len(offspring), kp.num_objects))]

    beta = 2 * np.random.rand() - 0.5 # between -1.5 and 1.5
    alpha = p1.alpha + beta * (p2.alpha - p1.alpha)
    
    ind = Individual(kp, alpha)
    ind.order = order
    
    return ind
    

def selection(kp: KnapsackProblem, population: list, k: int) -> Individual:
    indices = np.random.choice(range(len(population)), 5)
    selected = [population[idx] for idx in indices]
    fitnesses = [fitness(kp, s) for s in selected]
    max_idx = np.argmax(fitnesses)
    return selected[max_idx]

def elimination(kp: KnapsackProblem, population: list, offspring: list, lambda_: int) -> list:
    combined = population + offspring
    combined = sorted(combined, key=lambda x: fitness(kp, x), reverse=True)
    
    return combined[:lambda_]

## Main Evolutionary Algorithm

In [498]:
def initialize(kp: KnapsackProblem, lambda_: int) -> list:
    return [Individual(kp) for _ in range(lambda_)]


def evolutionaryAlgorithm(kp: KnapsackProblem,
                          lambda_: int,
                          mu: int,
                          num_iterations: int,
                          k: int):

    # initialize population
    population = initialize(kp, lambda_)
    
    # heuristic solution
    heur_order = np.argsort(kp.values/kp.weights)[::-1]
    heur_solution = Individual(kp)
    heur_solution.order = heur_order
    print(f'Heuristic best solution fitness = {fitness(kp, heur_solution)}')
    
    # intialize offspring list
    offspring = [None] * mu
    
    # run evolutionary algorithm
    for ii in range(num_iterations):
        for jj in range(mu):
            
            # select parents
            parent1 = selection(kp, population, k=k)
            parent2 = selection(kp, population, k=k)
            
            # generate offspring based on parents
            offspring[jj] = recombination(kp, parent1, parent2)
            
            # mutate offspring
            offspring[jj] = mutation(offspring[jj])
        
        # mutation on original population
        population = [mutation(ind) for ind in population]
        
        # elimination
        population = elimination(kp, population, offspring, lambda_=lambda_)
        
        # calculate fitnesses
        fitnesses = [fitness(kp, ind) for ind in population]
        
        print(f'Mean fitness: {sum(fitnesses)/len(fitnesses)}, ', end='')
        print(f'Best fitness: {max(fitnesses)}')
        

In [499]:
num_objects = 50
lambda_ = 100  # population size
mu = 100  # offspring size
k = 5
num_iterations = 25

kp = KnapsackProblem(num_objects)

evolutionaryAlgorithm(kp, lambda_, mu, num_iterations, k)

Heuristic best solution fitness = 25.44608088689084
Mean fitness: 20.519892133802067, Best fitness: 22.784261128211668
Mean fitness: 21.937706430647196, Best fitness: 24.048901012177705
Mean fitness: 22.6417156947725, Best fitness: 24.588703778614388
Mean fitness: 23.470915175765004, Best fitness: 24.588703778614388
Mean fitness: 24.0125904584709, Best fitness: 24.749884898132553
Mean fitness: 24.353075962921945, Best fitness: 25.062880052301058
Mean fitness: 24.554817376922955, Best fitness: 25.107183599503397
Mean fitness: 24.708715010436194, Best fitness: 25.28918836557015
Mean fitness: 24.824261986540055, Best fitness: 25.28918836557015
Mean fitness: 24.96218909844758, Best fitness: 25.28918836557015
Mean fitness: 25.094656354241334, Best fitness: 25.441764835314668
Mean fitness: 25.15612503421458, Best fitness: 25.441764835314675
Mean fitness: 25.199421870077842, Best fitness: 25.441764835314675
Mean fitness: 25.28493313949829, Best fitness: 25.441764835314675
Mean fitness: 25.368

### TODO
Improve methods to consistently outperform the heuristic for problems of size 250-500.